In [1]:
def get_prompt(english_input: str) -> str:
    return f"""User:The motor has a broken piston. Let's get a repairman to fix it.

    Assistant:個摩打有條活塞爆咗，揾個師傅嚟整整佢。<｜end▁of▁sentence｜>User:He has a husky, magnetic voice.

    Assistant:佢把聲沙沙哋、沉沉哋，好有磁性。<｜end▁of▁sentence｜>User:A sugar cube is about the same amount as a teaspoon.

    Assistant:一粒方糖差唔多係一茶匙糖咁滯。<｜end▁of▁sentence｜>User:{english_input}

    Assistant:"""

In [6]:
import subprocess

def infer(english_input: str) -> str:
    prompt = get_prompt(english_input)
    result = subprocess.run([
        "llama.cpp/main",
        "-t", "10",
        "-ngl", "64",
        "-b", "512",
        "-m", "models/deepseek-llm-7b-chat.Q5_K_M.gguf",
        "--seed", "1",
        "--temp", "0",
        "--repeat_penalty", "1.1",
        "-n", "-1",
        "-p", prompt
    ], capture_output=True, text=True)
    return result.stdout.split("\n\n    Assistant:")[-1]


In [7]:
infer("I am a student.")

'我係個學生。'

# Start the Test

In [8]:
from tqdm import tqdm

In [10]:
with open("wordshk-parallel-corpus/plus15/test.en.txt", "r") as test_file:
    test_lines = test_file.readlines()

with open("results-3-shot.txt", "w") as results_file:
    for line in tqdm(test_lines):
        result = infer(line.strip())
        # Escape newlines just in case to not mess up line counts
        results_file.write(result.replace("\n", "\\n") + "\n")
        results_file.flush()

100%|██████████| 1500/1500 [47:21<00:00,  1.89s/it] 
